# 根据专家词库进行二次样本筛选

In [1]:
import pandas as pd
import re
import numpy as np
label2id = {
    '非常不满-服务敏感':0,
    '非常不满-费用敏感':1,
    '非常不满-渠道敏感':2,
    '一般不满':3,
    '比较不满':4
}
# 渠道敏感>费用敏感>服务敏感

    非常不满-服务敏感':0
    非常不满-费用敏感':1
    非常不满-渠道敏感':2

In [31]:
df_data = pd.read_csv('./data.csv')[['情绪等级标注结果','受理内容']]
df_data.columns = ["label","content"]
data = df_data.copy()
data.columns=["label","content"]
data0 = np.asarray(pd.read_csv('./服务敏感.csv')).squeeze()
data1 = np.asarray(pd.read_csv('./费用敏感.csv')).squeeze()
data2 = np.asarray(pd.read_csv('./渠道敏感.csv')).squeeze()

In [32]:
data = data.dropna()
data.label.value_counts()

比较不满         15932
非常不满-渠道敏感     6222
一般不满          4111
非常不满-服务敏感     2385
非常不满-费用敏感     1397
Name: label, dtype: int64

In [33]:
data['label'] = data['label'].apply(lambda x:label2id.get(x))
data.head()

,label,content
0,2,19927044318来电反映18026183816号码要求挂失，已和用户解释，该号码实时信...
1,2,【费用争议帐期】0\r\n【争议上网时间】\r\n【争议金额(元)】\r\n用户来电表示其宽...
2,2,【费用争议帐期】2019-08-26\r\n【争议上网时间】2019-08-26 18:48...
3,2,(总经理热线)客户对前单处理结果不满，工单编号：2019082310011783 已归档）\...
4,2,【接入方式】FTTH(光纤到户)\r\n【故障单号】20190826000031010\r\...


In [34]:
data['content'] = data['content'].apply(lambda x:re.sub('[0-9][0-9]{5,}|[\\n]|[/\\r]|[【]|[】]|[[]|[]]|[A-Za-z0-9_]{5,}|[,]','',x))
data['content'] = data['content'].apply(lambda x:re.sub('([0-9]{3}[1-9]|[0-9]{2}[1-9][0-9]{1}|[0-9]{1}[1-9][0-9]{2}|[1-9][0-9]{3})-(((0[13578]|1[02])-(0[1-9]|[12][0-9]|3[01]))|((0[469]|11)-(0[1-9]|[12][0-9]|30))|(02-(0[1-9]|[1][0-9]|2[0-8])))|[0-9]{4}[年][0-9]{1,}[月][0-9]{1,}[日]|[0-9]{1,}[:0-9]{1,}[:0-9]{1,}','',x))
# bert allow max_len == 512 包括[CLS][SEP]
data['content'] = data['content'].apply(lambda x:x[:510] if len(x)>510 else x)
data.head()

,label,content
0,2,来电反映号码要求挂失，已和用户解释，该号码实时信用额度停机，需预存费用复机后再挂失，或是直接...
1,2,费用争议帐期0争议上网时间争议金额(元)用户来电表示其宽带为何现在又欠费停机了，前台经核实，...
2,2,费用争议帐期争议上网时间 争议金额(元)客户投诉内容客户投诉号码 在 维 期间发生费用，认...
3,2,(总经理热线)客户对前单处理结果不满，工单编号： 已归档）前工单反馈内容：（用户来电反映其手...
4,2,接入方式FTTH(光纤到户)故障单号故障现象无法连接上网故障申告时间 是否为时限内第3次催修...


    渠道敏感>费用敏感>服务敏感
    非常不满-服务敏感':0
    非常不满-费用敏感':1
    非常不满-渠道敏感':2

In [35]:
data_arr = np.asarray(data)
len(data_arr)

30047

In [36]:
for i in range(len(data)):
    for corpus in data2:
        if corpus in data_arr[i][1]:
            data_arr[i][0] = 2
            continue
    for corpus in data1:
        if corpus in data_arr[i][1]:
            data_arr[i][0] = 1
            continue
    for corpus in data0:
        if corpus in data_arr[i][1]:
            data_arr[i][0] = 0
            continue

In [44]:
data_new = pd.DataFrame(data_arr,columns=['label','columns'])
data_new.label.value_counts()

0    18893
4     6057
3     2352
2     2012
1      733
Name: label, dtype: int64

In [38]:
data_0_5000 = data_new[:5000]

array([[0,
        '费用争议帐期争议上网时间 争议金额(元)客户投诉内容客户投诉号码  在 维 期间发生费用，认为不合理，要求处理。客户对 9月 计费月份产生的话费有异议。系统状态核查经查 CRM 系统显示 维  受理日期   故障恢复  日期 至今仍未修复 受影响的月租金额为  元月 元。前台预处理情况已向客户解释 但不接受，意见极大 客户要求户意见大，表示因故障造成其损失很大，要求我司专人回复给予合理解释以及退费！ '],
       [0,
        '(总经理热线)客户对前单处理结果不满，工单编号： 已归档）前工单反馈内容：（用户来电反映其手机号码（）在8月20号有接到一万号的来电说其号码有个低消27元就可以赠送一条宽带的优惠，引导用户其营业厅办理，但是用户去到营业厅后工作人员告知没有这个优惠活动，用户觉得被欺骗了，要求投诉）前工单归档内容：（我中心向用户了解，用户称已有工作人员跟进处理好，用户清楚）本次来电争议问题：表示问题没有人联系处理，请加急协助处理，谢谢'],
       [0,
        '接入方式FTTH(光纤到户)故障单号故障现象无法连接上网故障申告时间 是否为时限内第3次催修障否客户要求修障时间无客户其他要求终端类型E8-C热线接入：用户反映本机宽带出现无法正常上网，经查询当前为严障状态，拦截时间范围：从 时间开始到 时间结束，另在 已有在途故障工单号：，已向户解释当前故障现象并按解释口径向户解释，及建议户耐心等待，经解释多次无效，客户经理强烈要求前台为户下投诉工单催修障处理，请加紧处理，谢谢。'],
       ...,
       [0,
        '客户经理姓名工号无发生时间 （总经理热线）客户对前单处理结果不满，工单编号： （已归档）前工单反馈内容：现用户再次反馈问题，称该问题还没有解决，现要求我司继续核实处理，用户意见很大，，前台经安抚解释无效，请紧急协助处理，谢谢！ 前工单归档内容：  联系 翟小姐了解情况，用户称有工作人员联系回复说15天处理，经向其解释：工作人员在复大约15-1个月左右处理，到时再联系，用户清楚，录音编码： 本次来电争议问题：用户称当时说会帮其全部减免，为何现在查询还有欠费，用户对此表示不满要求我司尽快联系处理好，前台经解释无效，请协助处理。谢谢！'],
     

In [45]:
data_new = data_new[data_new['label']!=0]
data_new

,label,columns
0,2,来电反映号码要求挂失，已和用户解释，该号码实时信用额度停机，需预存费用复机后再挂失，或是直接...
1,2,费用争议帐期0争议上网时间争议金额(元)用户来电表示其宽带为何现在又欠费停机了，前台经核实，...
5,2,用户通过欢GO ，反馈他 他的宽带要支持IPV6。前台咨询用户是否是要修改成公网IP或者私网...
8,2,服务点营业厅名称或地址松山湖营业厅客服代表工号事件发生时间 来电反映其前往松山湖营业厅办理打...
9,2,接入方式FTTH(光纤到户)故障单号故障现象无法连接上网故障申告时间 是否已有工作人员联系 ...
...,...,...
30037,3,订单编码无订单环节信息移机或装机新装客户要求完成时间 客户其他需求接入方式终端类型IM号: ...
30038,4,用户来电反映其身份证：宽带办理注销，接入号：，但用户称其之前宽带有个预存元返还的费用还未使用...
30041,4,区县 城区路名 欧浦御龙湾-C5栋用户来电反映本机号码在韶关 城区 - 欧浦御龙湾-C8栋电...
30045,3,区县 长安路名 大岭山杨屋第三工业区用户来电反映其手机（）上网信号只有2G信号，前台查询地址...


In [47]:
data_new = list(np.asarray(data_new[data_new['label']!=0]))
for i in data_0_5000:
    data_new.append(i)
len(data_new)

16154

In [51]:
data_new = np.asarray(data_new)

In [52]:
import pickle
with open('./data_new.pkl','wb') as f:
    pickle.dump(data_new,f)
f.close()

In [53]:
df_data.label.value_counts()

比较不满         15932
非常不满-渠道敏感     6222
一般不满          4111
非常不满-服务敏感     2385
非常不满-费用敏感     1397
Name: label, dtype: int64